In [ ]:
!pip install transformers accelerate seqeval datasets>=1.8.0 torch>=1.3

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
# tokenizer
from transformers import AutoTokenizer, DistilBertTokenizerFast, BertTokenizerFast
# sequence tagging model + training-related
from transformers import DistilBertForTokenClassification, Trainer, TrainingArguments, BertForTokenClassification
import numpy as np
import pandas as pd
import torch
import json
import sys
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the data from Google drive

In [ ]:
# mount Google Drive
from google.colab import drive
import os

drive.mount("/content/drive", force_remount=True)

# data location
#parent_dir = "/content/drive/MyDrive"
parent_dir = "/content/drive/MyDrive/"
mk_data_path = lambda filename: os.path.join(parent_dir, filename)
#data_path = f"{parent_dir}/BERT_data_final.xlsx"
data_path = mk_data_path("BERT_data_final.xlsx")

# once finished, flush changes to files to make them visible (assuming we write to Google Drive)
# drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
# # mount Google Drive
# from google.colab import drive
# import os

# drive.mount("/content/drive", force_remount=True)

# # data location
# #parent_dir = "/content/drive/MyDrive"
# parent_dir = "/content/drive/MyDrive/"
# mk_data_path = lambda filename: os.path.join(parent_dir, filename)
# #data_path = f"{parent_dir}/BERT_data_final.xlsx"
# data_path = mk_data_path("final_changed_data.xlsx")

# # once finished, flush changes to files to make them visible (assuming we write to Google Drive)
# # drive.flush_and_unmount()

# Tokenizer setup

BERT relies on a wordpiece tokenization strategy that makes use of a set of special tokens used for prediction.

In [ ]:
model_name = "bert-large-uncased" # "distilbert-base-uncased", distilbert-base-cased
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name) #Distil
# convenience function for wordpiece tokenization of a list of tokens
tokenize = lambda ds: tokenizer(ds, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
RANDOM_SEED = 42

In [ ]:
# special tokens
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [ ]:
#no determiners data
# data = pd.read_excel(data_path)
# data.head(4)

In [ ]:
#data

In [ ]:
data = pd.read_excel(data_path)
data.head(4)

,Causal-relation text,BIO Code,"{""words"":Causal-relation text,""ner"":BIO Code}",BIO Code_changed,BIO Code-changed-NER
0,"[""When "",""a "",""policyholder "",""or "",""insured ""...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""When "",""a "",""policyholder "",""or "",""...","[""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."
1,"[""During "",""2020 "",""in "",""response "",""to "",""th...","[""O"",""O"",""O"",""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""During "",""2020 "",""in "",""response "",...","[""O"",""O"",""O"",""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""...","[""O"",""O"",""O"",""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-..."
2,"[""Prolonged "",""periods "",""of "",""low "",""interes...","[""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT...","{""words"":[""Prolonged "",""periods "",""of "",""low ""...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""CT"",""E"",""E"",""O"",...","[""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""..."
3,"[""Conversely "",""a "",""rise "",""in "",""interest "",...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT"",""O...","{""words"":[""Conversely "",""a "",""rise "",""in "",""in...","[""O"",""O"",""C"",""C"",""C"",""C"",""O"",""CT"",""O"",""O"",""O"",...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data[["Causal-relation text", "BIO Code"]].rename(columns={"Causal-relation text": "X", "BIO Code": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.loads(tokens) for tokens in dataset.X.values], [json.loads(labels) for labels in dataset.y.values]

In [ ]:
# load data and rename relevant columns to X (input) and y (output) # no determiners data
dataset = data[["Causal-relation text", "BIO Code_changed"]].rename(columns={"Causal-relation text": "X", "BIO Code_changed": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.loads(tokens) for tokens in dataset.X.values], [json.loads(labels) for labels in dataset.y.values]

In [ ]:
unique_tags = set(label for labels in y_raw for label in labels)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
print(id2tag)

{0: 'O', 1: 'B-E', 2: 'I-E', 3: 'B-C', 4: 'I-CT', 5: 'B-CT', 6: 'I-C'}


Split the data into train, dev using train_test_split. Further divide the train data into train and test data using k-fold cross validation.

In [ ]:
# Our data is split into sentences


# FIXME: we also need test data!
X_train_raw, X_dev_raw, y_train_raw, y_dev_raw = train_test_split(X_raw, y_raw, test_size=.2, random_state=RANDOM_SEED)

In [ ]:
X_train_final = np.array(X_train_raw)
y_train_final = np.array(y_train_raw)

<ipython-input-26-100920d9c2b5>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_final = np.array(X_train_raw)
<ipython-input-26-100920d9c2b5>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_train_final = np.array(y_train_raw)


In [ ]:
print(np.shape(X_dev_raw))
print(np.shape(y_dev_raw))
print(np.shape(X_train_raw))


(447,)
(447,)
(1787,)


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:2007: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [ ]:
import os
k=2
seed = 1
kf = KFold(n_splits=k,random_state=seed, shuffle=True)
metrics=[]
i=0
for train_index, val_index in kf.split(X_train_final,y_train_final):
  train_texts, test_texts =X_train_final[train_index].tolist(),X_train_final[val_index].tolist()
  train_labels, test_labels = y_train_final[train_index].tolist(),y_train_final[val_index].tolist()

In [ ]:
# print(type(test_labels))

In [ ]:
# X_train_raw

In [ ]:
# ensure our data is well-formed:
for ds in [X_train_raw, X_train_raw]:
  for row in ds:
    #print(row)
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# ensure our data is well-formed:
for ds in [X_dev_raw, X_dev_raw]:
  for row in ds:
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# ensure our data is well-formed:
for ds in [train_texts, train_texts]:
  for row in ds:
    assert all(len(tok) > 0 for tok in row)

In [ ]:
def encode_tags(tags, encodings, tag2id):
    """
    See https://huggingface.co/transformers/custom_datasets.html#tok-ner

    >If the tokenizer splits a token into multiple sub-tokens, then we will end up with a mismatch between our tokens and our labels.
    >One way to handle this is to only train on the tag labels for the first subtoken of a split token.
    >We can do this in 🤗 Transformers by setting the labels we wish to ignore to -100.
    >In the example above, if the label for @HuggingFace is 3 (indexing `B-corporation`),
    >we would set the labels of ['@', 'hugging', '##face'] to [3, -100, -100].
    """
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)
        # print(arr_offset)
        # print(type(arr_offset))

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

In [ ]:
# encoded X
X_train = tokenize(train_texts)
X_dev = tokenize(X_dev_raw)
X_test= tokenize(test_texts)

y_train = encode_tags(tags=train_labels, encodings=X_train, tag2id=tag2id)
y_dev = encode_tags(tags=y_dev_raw, encodings=X_dev, tag2id=tag2id)
y_test = encode_tags(tags=test_labels, encodings=X_test, tag2id=tag2id)


for X in [X_train, X_dev, X_test]:
  X.pop("offset_mapping")

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    """
    Pytorch dataset for easy batching
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_dev, y_dev)
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=y_train)

In [ ]:
training_args = TrainingArguments(
    output_dir=mk_data_path("results"),   # output directory
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags)) #Distil

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-large

Epoch,Training Loss,Validation Loss
1,1.546900,1.448925
2,1.258600,1.171272
3,0.913800,0.696015


In [ ]:
#Predictions from the model

In [ ]:
preds

array([[2, 2, 2, ..., 6, 6, 6],
       [0, 2, 2, ..., 2, 6, 6],
       [2, 2, 2, ..., 2, 2, 2],
       ...,
       [2, 6, 6, ..., 2, 2, 2],
       [2, 0, 6, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 6, 6]])

In [ ]:
preds.shape

(893, 125)

In [ ]:
np.shape(y_test)

(893, 125)

In [ ]:
flat_list = [item for sublist in preds for item in sublist]

In [ ]:
np.shape(flat_list)

(111625,)

In [ ]:
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

In [ ]:
np.shape(flat_list1)

(111625,)

In [ ]:
flat_list_testtexts = [(ind,item1) for sublist1 in test_texts for ind,item1 in enumerate(sublist1)]

In [ ]:
flat_list_testtexts_frame = pd.DataFrame(flat_list_testtexts)

In [ ]:
flat_list_testtexts_frame

,0,1
0,0,Employee
1,1,compensation
2,2,and
3,3,benefits
4,4,increased
...,...,...
29669,26,to
29670,27,the
29671,28,increased
29672,29,capital


In [ ]:
flat_list_testtexts_frame.columns = ['index','texts']

In [ ]:
flat_list_testtexts_frame

,index,texts
0,0,Employee
1,1,compensation
2,2,and
3,3,benefits
4,4,increased
...,...,...
29669,26,to
29670,27,the
29671,28,increased
29672,29,capital


In [ ]:
df1 = flat_list_testtexts_frame[flat_list_testtexts_frame.texts != ' ']

In [ ]:
df1

,index,texts
0,0,Employee
1,1,compensation
2,2,and
3,3,benefits
4,4,increased
...,...,...
29669,26,to
29670,27,the
29671,28,increased
29672,29,capital


In [ ]:
df1.to_excel('/content/text.xlsx')

In [ ]:
#Overall accuracy for the entire test data. Flatten the list and compare predicted with the gold label.

In [ ]:
type(test_texts)

list

In [ ]:
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)

In [ ]:
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)

In [ ]:
frame_combine_test.columns = ['preds','gold_data']

In [ ]:
frame_combine_test

,preds,gold_data
0,2,-100
1,2,1
2,2,2
3,2,2
4,2,2
...,...,...
111620,2,-100
111621,2,-100
111622,2,-100
111623,6,-100


In [ ]:
frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]

In [ ]:
frame_combine_test_new

,preds,gold_data
1,2,1
2,2,2
3,2,2
4,2,2
5,2,2
...,...,...
111528,4,4
111529,6,0
111530,6,3
111531,6,6


In [ ]:
targetname = ['I-E', 'B-E', 'I-C','I-CT', 'B-CT', 'B-C', 'O']

In [ ]:

from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

         I-E       0.71      0.91      0.80     10330
         B-E       1.00      0.00      0.00       986
         I-C       0.70      0.87      0.77     10340
        I-CT       0.63      0.70      0.66       866
        B-CT       0.74      0.67      0.70       914
         B-C       0.83      0.02      0.04      1022
           O       0.64      0.24      0.34      5216

    accuracy                           0.70     29674
   macro avg       0.75      0.49      0.48     29674
weighted avg       0.71      0.70      0.65     29674



In [ ]:
frame_combine_test_new.to_excel('/content/bert-large.xlsx')

In [ ]:
targetname = ['CT', 'C', 'O','E']

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

          CT       0.79      0.92      0.85      1780
           C       0.76      0.87      0.81     11362
           O       0.70      0.20      0.31      5216
           E       0.78      0.91      0.84     11316

    accuracy                           0.77     29674
   macro avg       0.76      0.72      0.70     29674
weighted avg       0.76      0.77      0.73     29674



In [ ]:
####################################################################################